In [10]:
import pandas as pd
from glob import glob
import os
import nltk
import ast # for converting string to literal representation
import json
import re
import datetime

In [11]:
datadir = "C:\\Users\\keatu\\Regis_archive\\practicum_data\\post_covid_symptoms_comments"
csv_list = glob(os.path.join(datadir,"*.csv"))

In [12]:
allcsv = []
for i in csv_list:
    print("Reading {}".format(i))
    allcsv.append(pd.read_csv(i))
fbdf = pd.concat(allcsv)

Reading C:\Users\keatu\Regis_archive\practicum_data\post_covid_symptoms_comments\fb_scraper_2022-2-6_16-17-21.csv
Reading C:\Users\keatu\Regis_archive\practicum_data\post_covid_symptoms_comments\fb_scraper_2022-2-6_17-21-13.csv


C:\Users\keatu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [7]:
fbdf.shape

(268, 56)

In [8]:
fbdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 0 to 143
Data columns (total 56 columns):
Unnamed: 0                       268 non-null int64
available                        268 non-null float64
comments                         268 non-null float64
comments_full                    268 non-null object
factcheck                        0 non-null float64
header                           256 non-null object
image                            48 non-null object
image_id                         18 non-null float64
image_ids                        268 non-null object
image_lowquality                 65 non-null object
images                           268 non-null object
images_description               268 non-null object
images_lowquality                268 non-null object
images_lowquality_description    268 non-null object
is_live                          268 non-null float64
likes                            268 non-null float64
link                             33 non-null ob

In [43]:
print("Unique column counts")
for i in fbdf.columns:
    print("{}: {}".format(i, len(fbdf[i].unique())))

Unique column counts
Unnamed: 0: 2307
available: 1
comments: 413
comments_full: 1
factcheck: 1
header: 964
image: 845
image_id: 210
image_ids: 210
image_lowquality: 1889
images: 846
images_description: 148
images_lowquality: 1890
images_lowquality_description: 154
is_live: 1
likes: 446
link: 1870
links: 3065
page_id: 34
post_id: 2206
post_text: 2542
post_url: 2167
reaction_count: 1
reactions: 1
reactors: 1
shared_post_id: 62
shared_post_url: 62
shared_text: 693
shared_time: 112
shared_user_id: 40
shared_username: 38
sharers: 1
shares: 99
text: 2691
time: 2232
timestamp: 2205
user_id: 923
user_url: 4283
username: 922
video: 53
video_duration_seconds: 1
video_height: 1
video_id: 29
video_ids: 4
video_quality: 1
video_size_MB: 1
video_thumbnail: 43
video_watches: 1
video_width: 1
videos: 4
w3_fb_url: 1
was_live: 2
with: 5339


In [70]:
remove_cols = ['Unnamed: 0', 'available', 'factcheck',
       'image', 'image_id', 'image_ids', 'image_lowquality',
       'images', 'images_description', 'images_lowquality',
       'images_lowquality_description', 'is_live',
       'reactions', 'reactors',
       'sharers', 'shares', 'video', 'video_duration_seconds', 'video_height',
       'video_id', 'video_ids', 'video_quality', 'video_size_MB',
       'video_thumbnail', 'video_watches', 'video_width', 'videos',
       'w3_fb_url', 'was_live', 'with']

In [72]:
# remove duplicates based on smallest null count value in each row and remove null columns
fbdf_trim = fbdf.copy()
fbdf_trim["null_count"] = fbdf_trim.count(axis=1)
fbdf_trim = fbdf_trim.sort_values(["null_count"])
fbdf_trim = fbdf_trim.groupby(["text"]).aggregate('first').reset_index()
fbdf_trim = fbdf_trim.drop(columns=remove_cols)
print(fbdf_trim.shape)

(2690, 24)


In [75]:
fbdf_trim.columns

Index(['text', 'comments', 'comments_full', 'header', 'likes', 'link', 'links',
       'page_id', 'post_id', 'post_text', 'post_url', 'reaction_count',
       'shared_post_id', 'shared_post_url', 'shared_text', 'shared_time',
       'shared_user_id', 'shared_username', 'time', 'timestamp', 'user_id',
       'user_url', 'username', 'null_count'],
      dtype='object')

In [76]:
fbdf_trim.to_csv(os.path.join(datadir,"trimmed_fb_data.csv"))

In [94]:
fbdf_trim

NameError: name 'fbdf_trim' is not defined

In [13]:
comments = 'C:\\Users\\keatu\\Regis_archive\\practicum_data\\covid-19_long_haulers_support_comments'
commentlist = glob(os.path.join(comments,"*.csv"))

In [14]:
cdf = pd.read_csv(commentlist[1])

In [17]:
def reform_comment(text):
    date_re = re.compile("datetime.datetime(\(.*?\))")
    dates = re.findall("datetime.datetime\(.*?\)", text)
    for date in dates:
        time_obj = eval(date)
        formatted = "'{}-{}-{} {}:{}:{}'".format(time_obj.year,time_obj.month,time_obj.day,time_obj.hour,time_obj.minute,time_obj.second)
        text = text.replace(date,formatted)
    text = text.replace("None","[]")
    return ast.literal_eval(text)

cdf["comments_full"] = cdf["comments_full"].apply(reform_comment)

In [32]:
test = {"x":1,"y":2}
test.pop("l",None)

In [33]:
test

{'x': 1, 'y': 2}

In [23]:
def comment_dfs(comment_list, post_id):
commentsdf = pd.DataFrame()
repliesdf = pd.Dataframe()
for comment in cdf["comments_full"][3]:
    if "replies" in comment:
        for reply in
    comment.pop("replies",None)
    comment["parent_id"] = post_id
    commentsdf.append(comment, ignore_index = True)

[{'comment_id': '449402106400544',
  'comment_url': 'https://facebook.com/449402106400544',
  'commenter_id': '1573784128',
  'commenter_url': 'https://facebook.com/sheri.angell?groupid=373920943948661&refid=18&__tn__=R',
  'commenter_name': 'Sheri Lynn Barbour',
  'commenter_meta': [],
  'comment_text': 'This is the first time on this page I have seen so many people having this issue. I have been posting about this. I had covid back in Nov and about a month ago started out of the blue with a rash which started on my back. The dermatologist just said I was having a allergic reaction to something. The rash has cleared up for the most part but I still have a few places with itchy bumps and just itchy all over and places feel tingly and kinda like a tingly burn at times. The pic I am posting is the first day it just appeared and it was on my back then went to other places .',
  'comment_time': '2021-2-2 21:58:15',
  'comment_image': 'https://scontent.fapa1-2.fna.fbcdn.net/v/t1.6435-9/cp0/

In [21]:
comment_df = pd.DataFrame
for item in cdf["commments_full"]:


[{'comment_id': '659369548737131',
  'comment_url': 'https://facebook.com/659369548737131',
  'commenter_id': '706990933',
  'commenter_url': 'https://facebook.com/david.spychalla?groupid=373920943948661&refid=18&__tn__=R',
  'commenter_name': 'David Spychalla',
  'commenter_meta': [],
  'comment_text': 'Thank you! The encouragement helps me to keep battling. I appreciate it! Glad your doing well.',
  'comment_time': '2022-2-2 21:58:3',
  'comment_image': [],
  'comment_reactors': [],
  'comment_reactions': [],
  'comment_reaction_count': [],
  'replies': []},
 {'comment_id': '659365875404165',
  'comment_url': 'https://facebook.com/659365875404165',
  'commenter_id': '1081494031',
  'commenter_url': 'https://facebook.com/colette.tihistalongin?groupid=373920943948661&refid=18&__tn__=R',
  'commenter_name': 'Colette Tihista-Longin',
  'commenter_meta': [],
  'comment_text': 'You indeed give me hope! Thank you! Im so happy for you!!!',
  'comment_time': '2022-2-2 21:52:3',
  'comment_ima

In [151]:
def ascii_encode_dict(data):
    ascii_encode = lambda x: x.encode('ascii','ignore')
    return dict(map(ascii_encode, pair) for pair in data.items())

JSONDecodeError: Expecting value: line 1 column 274 (char 273)

In [155]:
json.loads(comment)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)